# Setup

## Install packages

In [1]:
%%time
!pip install faiss-gpu
# install einops for embedding model
!pip install einops
# transformer version required for Embedding model
!pip uninstall -y transformers
!pip install transformers==4.42.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
Found existing installation: transformers 4.45.1
Uninstalling transformers-4.45.1:
  Successfully uninstalled transformers-4.45.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 64.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
CPU times: user 670 ms, sys: 154 ms, total: 824 ms
Wall time: 51.5 s


## Some configuration

In [4]:
# In DEBUG mode, infer only on 5 problems
DEBUG = False
# Number of candidate solutions to generate
K = 4
# Maximum number of turns
DEPTH = 4
# Generation Parameters
TEMPERATURE = 0.5
TOP_P = 0.9
# vLLM batch size
BATCH_SIZE = 64

## Imports

In [2]:
import re
import csv
import torch
import gc
from tqdm import tqdm
import pandas as pd
from queue import Queue, Empty
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager
import torch.nn.functional as F
from transformers import AutoModel
import faiss
import faiss.contrib.torch_utils

# Load Embedding Model

### We load the NV-Embed-v2 model from Nvidia. You can experiment with other models.

In [3]:
model = AutoModel.from_pretrained(
    "nvidia/NV-Embed-v2", trust_remote_code=True, torch_dtype=torch.bfloat16, # load in 16-bit precision
    device_map="auto"
)

config.json:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

configuration_nvembed.py:   0%|          | 0.00/3.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nvidia/NV-Embed-v2:
- configuration_nvembed.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_nvembed.py:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nvidia/NV-Embed-v2:
- modeling_nvembed.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Instruction added to each query

In [6]:
instruction = "Given a mathematical problem in Bengali, retrieve the problem most similar to the given problem."
prefix = "Instruct: " + instruction + "\nQuery: "

# Functions to generate embeddings

In [7]:
def embed_query(query):
    query_embeddings = model.encode(
        [query], instruction=prefix, max_length=4096
    )
    # normal embeddings so that we can dot product to get cosine distance
    query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
    return query_embeddings[0]

def embed_document(document):
    document_embeddings = model.encode(
        [document], instruction='', max_length=4096
    )
    document_embeddings = F.normalize(document_embeddings, p=2, dim=1)
    return document_embeddings[0]

# Load Datasets

In [4]:
example_df = pd.read_csv('/kaggle/input/math-bangla/Math_in_Bangla.csv')
example_df

,problem,solution
0,"""CORPORBTION"" শব্দের অক্ষর ব্যবহার করে কয়টি ...","'কর্পোরেশন' শব্দে, আমরা স্বরবর্ণ OOAIO কে একটি..."
1,"""CRYPTO"" কীওয়ার্ড সহ Playfair সাইফার বিবেচনা...","প্রথমে, আসুন ""CRYPTO"" কীওয়ার্ড ব্যবহার করে 5x..."
2,"""ENTRANCE"" শব্দ থেকে 4টি অক্ষর নিয়ে নির্বাচন...",আমাদের 8টি অক্ষর রয়েছে যার থেকে 6টি অনন্য। 4ট...
3,"""MATH"" কীওয়ার্ড ব্যবহার করে নিম্নলিখিত Vigen...","Vigenere সাইফার ডিক্রিপ্ট করতে, আমাদের এনক্রিপ..."
4,"""__45"" আকারে কতটি ধনাত্মক চার-অঙ্কের পূর্ণসংখ...","একটি সংখ্যা 45 দ্বারা বিভাজ্য হওয়ার জন্য, এটি..."
...,...,...
126670,√{(3.5)(10^9)} নিচের কোনটির সবচেয়ে কাছাকাছি? ...,√{(3.5)(10^9)} = √{(35)(10^8)} = √(35)√(10^8) ...
126671,√{(8.2)(10^9)} নিচের কোনটির সবচেয়ে কাছাকাছি? ...,√{(8.2)(10^9)} = √{(82)(10^8)} = √(82)√(10^8) ...
126672,【হুয়ানহুয়ানের অ্যাডভেঞ্চার ইন দ্য এক্সপো পার...,(1) জোন ডি-র প্রবেশপথে সেই সময়ে নিরাপত্তা চেক...
126673,甲 এবং 乙 প্রত্যেকে চারটি কোর্সের মধ্যে থেকে দুট...,To find the number of ways in which 甲 and 乙 ca...


In [5]:
test_df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
test_df

,ID,Problem
0,0,একটি কেক-কে সরলরৈখিকভাবে 2 বার কেটে সর্বোচ্চ ক...
1,1,একটি পুকুরের উপর 100 টি পাথর রাখা আছে। প্রথমে ...
2,2,"ধরো $f(x) = x^{67-x^{67-x^{67-\dots}}}$, যেখান..."
3,3,সামিন ও স্বর্গ গণনার জন্য শুধু 0 আর 1 ব্যবহার ...
4,4,"A, B, C তিনটি স্ট্যান্ড। A স্ট্যান্ড থেকে 2 টি..."
...,...,...
95,95,একটি বক্সে কিছু লাল এবং নীল বল রয়েছে। যদি আরও ...
96,96,একটি চার অঙ্কের পূর্ণসংখ্যা 11 দ্বারা নিঃশেষে ...
97,97,এমন কতগুলো ধনাত্মক পূর্ণসংখ্যা আছে যারা $12^{1...
98,98,"3 টি ক্রমিক পূর্ণসংখ্যার যোগফল 216 হলে, সবচেয়..."


## Sample 100 problems from the dataset

In [8]:
example_df = example_df.sample(100)
example_df

,problem,solution
89167,একটি ধনাত্মক পূর্ণসংখ্যা $n$ এবং অশূন্য অঙ্কগু...,$A_n = a(1 + 10 + \dots + 10^{n - 1}) = a \cdo...
14969,একটি x ব্যক্তির শ্রেণীকক্ষ 40% মেয়ে দ্বারা প...,"এই সমস্যাটি সমাধান করার জন্য, আমাদের x এর মান ..."
125252,স্কুল নাচের জন্য অর্থ সংগ্রহ করতে স্কুল শুরু হ...,"এই সমস্যাটি সমাধান করার জন্য, আমাদের x এর মান ..."
107646,"প্রাতঃরাশের জন্য, ডেইজি $2 এর জন্য একটি মাফিন ...","এই সমস্যাটি সমাধান করার জন্য, আমাদের x এর মান ..."
30577,জুন যদি 20টি সমান টাইল সমন্বিত একটি নকশা তৈরি...,জুন 20টি টাইলস নিয়ে একটি নকশা তৈরি করেছে। নকশ...
...,...,...
52589,ম্যাসির প্রতি সপ্তাহে মোট 24 মাইল দৌড়ানোর লক...,ম্যাসির লক্ষ্য প্রতি সপ্তাহে মোট 24 মাইল চালান...
73492,লিসা একটি গৃহহীন আশ্রয়কে দান করার জন্য মোজা ...,লিসা 12 জোড়া মোজা কিনেছে। সান্ড্রা 20 জোড়া ম...
13720,এক সপ্তাহের মধ্যে একটি 140-পৃষ্ঠার বই পড়া শে...,"এই সমস্যাটি সমাধান করার জন্য, আমাদের x এর মান ..."
7038,MLB-এর জন্য ক্যারিয়ার হোম রান লিডারদের মধ্যে...,আমরা জানি যে হ্যাঙ্ক অ্যারন 755 হোম রান করেছেন...


In [9]:
test_df = test_df.sample(10)
test_df

,ID,Problem
83,83,মণীষা একটি 256 পৃষ্ঠার নোটবুক কিনেছে। তুন্না প...
88,88,একটি গাছের জীবনের শুরুতে একটি পাতা থাকে। যেকোন...
7,7,"তিন বন্ধু শাহরিয়ার, সাকিব আর সাদিয়া। শাহরিয়..."
48,48,"তাহমিদের কাছে $1, 2, 3, 5, 7, 9$ সংখ্যা গুলো আ..."
13,13,যদি পায়েল তার মোবাইল ফোনে 3 অঙ্কবিশিষ্ট পাসওয়া...
51,51,$x$ ও $y$ দুইটি ধনাত্মক পূর্ণসংখ্যা। $xy = 196...
79,79,জ্যোতির পরীক্ষায় 6 টি প্রশ্ন এসেছে এবং তার কা...
29,29,একটি ধারার $n$ তম পদ হতে $(n - 1)$ তম পদের বিয়...
76,76,"একটি বৃত্তস্থ ট্রাপিজিয়াম আঁকা হলো, যার পরিবৃ..."
94,94,n সংখ্যক ক্রমিক পূর্ণসংখ্যার যোগফল n দ্বারা বি...


## 

## Convert the problems to convenient lists

In [10]:
problems = list(example_df['problem'])
solutions = list(example_df['solution'])
test_problems = list(test_df['Problem'])

In [11]:
problems[0]

'একটি ধনাত্মক পূর্ণসংখ্যা $n$ এবং অশূন্য অঙ্কগুলির জন্য $a$, $b$, এবং $c$, যাক $A_n$ হল $n$-অঙ্কের পূর্ণসংখ্যা যার প্রতিটি সংখ্যা $a$ এর সমান; যাক $B_n$ হল $n$-অঙ্কের পূর্ণসংখ্যা যার প্রতিটি সংখ্যা $b$ এর সমান, এবং $C_n$ হল $2n$-অঙ্ক ($n$-অঙ্ক নয়) পূর্ণসংখ্যা যার প্রতিটি সংখ্যা সমান $c$ থেকে $a + b + c$ এর সর্বশ্রেষ্ঠ সম্ভাব্য মান কি যার জন্য $n$ এর কমপক্ষে দুটি মান যেমন $C_n - B_n = A_n^2$? $\\textbf{(A)} \\text{ 12} \\qquad \\textbf{(B)} \\text{ 14} \\qquad \\textbf{(C)} \\text{ 16} \\qquad \\textbf{(D)} \\text{ 18} \\qquad \\textbf{(E)} \\text{ 20}$'

# Embed the Problems

In [12]:
embed_document(problems[0])

/root/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/0783263f3c009f67bd0e177040cfecad4b1171d6/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/opt/conda/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


tensor([-0.0063, -0.0251,  0.0151,  ..., -0.0117,  0.0017,  0.0271],
       device='cuda:0')

## Embed the database problems

In [13]:
problem_embeddings = []
for problem in tqdm(problems):
    embedding = embed_document(problem)
    problem_embeddings.append(embedding)
problem_embeddings = torch.stack(problem_embeddings)

100%|██████████| 100/100 [01:58<00:00,  1.19s/it]


## Embed the test problems

In [14]:
test_embeddings = []
for problem in tqdm(test_problems):
    embedding = embed_query(problem)
    test_embeddings.append(embedding)
test_embeddings = torch.stack(test_embeddings)

100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


## 

## Dimension of the embedding

In [15]:
dimension = problem_embeddings.shape[1] 
dimension

4096

# Create FAISS index on CPU

In [16]:
index = faiss.IndexFlatIP(dimension)
index.add(problem_embeddings.cpu().numpy()) 

# Find Similar Problems

In [17]:
# Top 5 most similar problems for now
top_k = 5
_, similar_indices = index.search(test_embeddings.cpu().numpy(), top_k)
similar_indices

array([[97, 11,  4,  5, 58],
       [ 0, 89, 11, 55, 93],
       [36,  0,  4, 89, 38],
       [ 0, 36,  4, 87, 29],
       [93, 36, 68, 29,  5],
       [ 0, 43, 38, 36, 93],
       [11,  4, 93,  0, 87],
       [ 0, 93, 36, 40, 87],
       [73,  0, 93, 57, 68],
       [ 0, 93, 38, 11, 36]])

# Create Prompts

In [18]:
prompts = [''] * len(test_problems)
for i, idx_list in enumerate(similar_indices): 
	# initial prompt
    prompt = "Here are a list of math problems and solutions:\n"
    # add examples
    for idx in idx_list:
        prompt += f"Problem: {problems[idx]}\nSolution: {solutions[idx]}\n\n" 
    # final problem
    prompt += "Now solve the problem below:\n"
    prompt += test_problems[i]
    prompts[i] = prompt

In [19]:
print(prompts[0])

Here are a list of math problems and solutions:
Problem:  এক সপ্তাহের মধ্যে একটি 140-পৃষ্ঠার বই পড়া শেষ করার পরিকল্পনা করার সময়, জেসি প্রাথমিকভাবে প্রতিদিন 3 বার, প্রতিবার, সপ্তাহের প্রতিটি দিন x পৃষ্ঠা পড়ার সিদ্ধান্ত নেয়। প্রকৃতপক্ষে তার লক্ষ্য অর্জনের জন্য তাকে প্রতিদিন 2টি পৃষ্ঠা পড়তে হবে। অজানা চলক x এর মান কত?
Solution: এই সমস্যাটি সমাধান করার জন্য, আমাদের x এর মান নির্ধারণ করতে হবে, যা জেসি প্রাথমিকভাবে প্রতিবার পড়ার সিদ্ধান্ত নিয়েছিল এমন পৃষ্ঠাগুলির সংখ্যাকে প্রতিনিধিত্ব করে। আমরা জানি যে জেসি প্রতিদিন 3 বার এবং প্রতিবার, সপ্তাহের প্রতিটি দিন x পৃষ্ঠা পড়ার পরিকল্পনা করে। অতএব, তিনি প্রতিদিন 3 * x = 3x পৃষ্ঠা পড়ার পরিকল্পনা করেছেন। আমরা এটাও জানি যে এক সপ্তাহের মধ্যে বইটি শেষ করার লক্ষ্য অর্জন করতে তাকে প্রতিদিন আরও 2টি পৃষ্ঠা পড়তে হবে। অতএব, তার প্রতিদিন 3x + 2 পৃষ্ঠা পড়া উচিত। বইটির মোট 140 পৃষ্ঠা রয়েছে এবং তিনি এটি এক সপ্তাহের মধ্যে শেষ করার পরিকল্পনা করেছেন। অতএব, তার প্রতিদিন 140/7 = 20 পৃষ্ঠা পড়া উচিত। আমরা নিম্নরূপ সমীকরণ সেট আপ করতে পারি: 3x + 2 = 20 আসুন x এ